# Introduction to Quantum Computing

## All About Qubits

### Codercise l.1.1 (Normalizeation of quantum state)

In [2]:
import numpy as np
import pennylane as qml

In [3]:
import numpy as np


def normalize_state(alpha, beta):
    """Compute a normalized quantum state given arbitrary amplitudes.

    Args:
        alpha (complex): The amplitude associated with the |0> state.
        beta (complex): The amplitude associated with the |1> state.

    Returns:
        np.array[complex]: A vector (numpy array) with 2 elements that represents
        a normalized quantum state.
    """
    # Compute the norm (magnitude) of the state vector
    norm = np.sqrt(np.abs(alpha)**2 + np.abs(beta)**2)

    # Normalize the amplitudes
    alpha_prime = alpha / norm
    beta_prime = beta / norm

    # Create and return the normalized state vector
    return np.array([alpha_prime, beta_prime])


# Example usage
alpha = 1 + 1j
beta = 1 - 1j
normalized_state = normalize_state(alpha, beta)
print(normalized_state)

[0.4472136  0.89442719]


$np.abs(alpha)**2$ computes the squared magnitude of alpha.
$np.abs(beta)**2$ computes the squared magnitude of beta.
$np.sqrt(...)$ computes the square root of the sum of the squared magnitudes of alpha and beta.
norm now holds the magnitude of the vector represented by alpha and beta.

### Codercise l.1.2 (Inner product & orthonormal base)

Complete the inner_product function below that computes the inner product between two arbitrary states. Then, use it to verify that |0> and |1> form an orthonormal basis, i.e., the states are normalized and orthogonal.

In [1]:
import numpy as np
def inner_product(state_1, state_2):
    """Compute the inner product between two states.

    Args:
        state_1 (np.array[complex]): A normalized quantum state vector
        state_2 (np.array[complex]): A second normalized quantum state vector

    Returns:
        complex: The value of the inner product <state_1 | state_2>.
    """

    # Compute the conjugate of state_1
    state_1_conj = np.conj(state_1)

    # Compute the element-wise product of state_1_conj and state_2
    product = state_1_conj * state_2
    # Sum the elements of the product
    inner_prod = np.sum(product)
    
    return inner_prod


# Test your results with this code
ket_0 = np.array([1, 0])
ket_1 = np.array([0, 1])

print(f"<0|0> = {inner_product(ket_0, ket_0)}")
print(f"<0|1> = {inner_product(ket_0, ket_1)}")
print(f"<1|0> = {inner_product(ket_1, ket_0)}")
print(f"<1|1> = {inner_product(ket_1, ket_1)}")


<0|0> = 1
<0|1> = 0
<1|0> = 0
<1|1> = 1


### Codercise I.1.3 — Sampling measurement outcomes

In [21]:
def measure_state(state, num_meas):
    """Simulate a quantum measurement process.

    Args:
        state (np.array[complex]): A normalized qubit state vector.
        num_meas (int): The number of measurements to take

    Returns:
        np.array[int]: A set of num_meas samples, 0 or 1, chosen according to the probability
        distribution defined by the input state.
    """

    # Compute the measurement outcome probabilities
    prob_0 = np.abs(state[0])**2
    prob_1 = np.abs(state[1])**2

  # Generate random numbers for each measurement
    random_numbers = np.random.rand(num_meas)

  # Determine the measurement outcomes based on probabilities
    measurements = np.where(random_numbers <= prob_0, 0, 1)

    return measurements
    pass

state = np.array([0.8, 0.6])
measure_state(state, 10)

array([0, 1, 0, 1, 0, 0, 0, 0, 0, 0])

### Codercise I.1.4 — Applying a quantum operation

In [24]:
import numpy as np

U = np.array([[1, 1], [1, -1]]) / np.sqrt(2)

def apply_u(state):
    """Apply a quantum operation.

    Args:
        state (np.array[complex]): A normalized quantum state vector.

    Returns:
        np.array[complex]: The output state after applying U.
    """
    # # Ensure the state is a column vector
    # state = state.reshape(-1, 1)
    
    # Apply the unitary matrix U to the input state
    new_state = np.dot(U, state.reshape(-1, 1))
    
    return new_state

# Example usage
initial_state = np.array([1, 0])  # |0⟩ state
output_state = apply_u(initial_state)
print("Output state:", output_state)

Output state: [[0.70710678]
 [0.70710678]]
[[1]
 [0]]


## Quantum Circuits

### Codercise I.2.1 — Order of operations

The code below is a quantum function with all the gates from the circuit shown below. However, the gates are out of order! Re-arrange the lines of the function to match the order of operations in the circuit.  

![image.png](https://assets.cloud.pennylane.ai/codebook/circuit_i-2-1.svg)

In [15]:
import numpy as np
import pennylane as qml

dev = qml.device("default.qubit", wires=3)
@qml.qnode(dev)
def my_circuit(theta, phi):
    ##################
    # YOUR CODE HERE #
    ##################

    # REORDER THESE 5 GATES TO MATCH THE CIRCUIT IN THE PICTURE

    qml.CNOT(wires=[0, 1])
    qml.RX(theta, wires=2)
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[2, 0])
    qml.RY(phi, wires=1)

    # This is the measurement; we return the probabilities of all possible output states
    # You'll learn more about what types of measurements are available in a later node
    return qml.probs(wires=[0, 1, 2])


In [17]:

drawer = qml.draw(my_circuit)
print(drawer(0.3, 0.1))

0: ─╭●─────────H─╭X───────────┤ ╭Probs
1: ─╰X───────────│───RY(0.10)─┤ ├Probs
2: ──RX(0.30)────╰●───────────┤ ╰Probs


### Codercise I.2.2 — Building a QNode

Recall that one way in which we can turn our quantum circuits into QNodes is via the qml.QNode function:

my_qnode = qml.QNode(my_circuit, my_device)

Once a QNode is created, it can be called like a function using the same parameters as the quantum function upon which it's built.

Complete the quantum function in the PennyLane code below to implement the following quantum circuit. Then, construct a QNode using qml.QNode and run the circuit on the provided device.

![images](https://assets.cloud.pennylane.ai/codebook/circuit_i-2-2.svg)

In [54]:
# This creates a device with three wires on which PennyLane can run computations
dev = qml.device("default.qubit", wires=3)

@qml.qnode(dev)
def my_circuit(theta, phi, omega):

    ##################
    # YOUR CODE HERE #
    ##################

    # IMPLEMENT THE CIRCUIT BY ADDING THE GATES
    qml.RX(theta, wires=0)
    qml.RY(phi, wires=1)
    qml.RZ(omega, wires=2)
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[1, 2])
    qml.CNOT(wires=[2, 0])
    # Here are two examples, so you can see the format:
    # qml.CNOT(wires=[0, 1])
    # qml.RX(theta, wires=0)

    return qml.probs(wires=[0, 1, 2])

# We set up some values for the input parameters
theta, phi, omega = 0.1, 0.2, 0.3



In [60]:
# drawer = qml.draw(my_circuit)
print(qml.draw(my_circuit)(theta, phi, omega))

0: ──RX(0.10)─╭●────╭X─┤ ╭Probs
1: ──RY(0.20)─╰X─╭●─│──┤ ├Probs
2: ──RZ(0.30)────╰X─╰●─┤ ╰Probs


### Codercise I.2.3 — The QNode decorator

The second way to construct a QNode in PennyLane is using a decorator. Decorating a quantum function with **@qml.qnode(dev)** will automatically produce a QNode with the same name as your function that can be run on the device dev.

The quantum function below implements the circuit from the previous exercise. Apply a decorator to the quantum function to construct a QNode, then run it using the provided input parameters.

In [65]:
dev = qml.device("default.qubit", wires=3)

##################
# YOUR CODE HERE #
##################
@qml.qnode(dev)
# DECORATE THE FUNCTION BELOW TO TURN IT INTO A QNODE


def my_circuit(theta, phi, omega):
    qml.RX(theta, wires=0)
    qml.RY(phi, wires=1)
    qml.RZ(omega, wires=2)
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[1, 2])
    qml.CNOT(wires=[2, 0])
    return qml.probs(wires=[0, 1, 2])


theta, phi, omega = 0.1, 0.2, 0.3


print(qml.draw(my_circuit)(theta, phi, omega))

# RUN THE QNODE WITH THE PROVIDED PARAMETERS


0: ──RX(0.10)─╭●────╭X─┤ ╭Probs
1: ──RY(0.20)─╰X─╭●─│──┤ ├Probs
2: ──RZ(0.30)────╰X─╰●─┤ ╰Probs


### Codercise I.2.4 — Circuit depth

Remember our circuit from the previous section:

![image.png](https://assets.cloud.pennylane.ai/codebook/circuit_i-2-2.svg)

What is the depth of the circuit in the picture above?

In [3]:
dev = qml.device("default.qubit", wires=3)


@qml.qnode(dev)
def my_circuit(theta, phi, omega):
    qml.RX(theta, wires=0)
    qml.RY(phi, wires=1)
    qml.RZ(omega, wires=2)
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[1, 2])
    qml.CNOT(wires=[2, 0])
    return qml.probs(wires=[0, 1, 2])


##################
# YOUR CODE HERE #
##################

# FILL IN THE CORRECT CIRCUIT DEPTH
depth = 4

theta, phi, omega = 0.1, 0.2, 0.3


print(qml.draw(my_circuit)(theta, phi, omega))

0: ──RX(0.10)─╭●────╭X─┤ ╭Probs
1: ──RY(0.20)─╰X─╭●─│──┤ ├Probs
2: ──RZ(0.30)────╰X─╰●─┤ ╰Probs


## Unitary Matrices

### Codercise I.3.1 — Unitaries in PennyLane

In PennyLane, unitary operations specified by a matrix can be implemented in a quantum circuit using the QubitUnitary operation. QubitUnitary is a parametrized gate, and can be called like so:

**qml.QubitUnitary(U, wires=wire)**

Complete the quantum function below to create a circuit that applies U to the qubit and returns its state. (Compare this to the earlier function apply_u that you wrote before - isn't it nice to not have to worry about the matrix arithmetic?)

In [7]:
dev = qml.device("default.qubit", wires=1)

U = np.array([[1, 1], [1, -1]]) / np.sqrt(2)


@qml.qnode(dev)
def apply_u():

    ##################
    # YOUR CODE HERE #
    ##################

    # USE QubitUnitary TO APPLY U TO THE QUBIT
    qml.QubitUnitary(U, wires=0)
    # Return the state
    return qml.state()

print(qml.draw(apply_u)())

0: ──U(M0)─┤  State

M0 = 
[[ 0.70710678  0.70710678]
 [ 0.70710678 -0.70710678]]


### Codercise I.3.2 — Parametrized unitaries

Unitary matrices can be parametrized. A single-qubit unitary operation can be expressed in terms of just three real numbers:
![equation]()
In PennyLane, this parametrized operation is implemented as a gate called Rot. Rot takes three parameters, which are precisely the angles in the formula above:

**qml.Rot(phi, theta, omega, wires=wire)**

Apply the Rot operation to a qubit using the input parameters. Then, complete the QNode to return the quantum state vector, using qml.state().

In [8]:
dev = qml.device("default.qubit", wires=1)

@qml.qnode(dev)
def apply_u_as_rot(phi, theta, omega):
    """
    Applies a Rot gate using the provided input parameters.

    Args:
        phi (float): Rotation angle around the X axis.
        theta (float): Rotation angle around the Y axis.
        omega (float): Rotation angle around the Z axis.
        
    Returns:
        (np.tensor): The quantum state vector.
    """
    # Apply the Rot gate
    qml.Rot(phi, theta, omega, wires=0)
    
    # Return the quantum state vector
    return qml.state()

# Test the function
phi, theta, omega = 0.1, 0.2, 0.3
state_vector = apply_u_as_rot(phi, theta, omega)
print("Quantum state vector:", state_vector)


Quantum state vector: [0.97517033-0.19767681j 0.09933467+0.00996671j]
